<a href="https://colab.research.google.com/github/Herbet-Meneses/ETL_process/blob/main/ETL_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ETL**

Objetivos:

Executar processo ETL;

Extrair dados bancários e de capitalização de mercado do ficheiro JSON bank_market_cap.json;

Transformar a moeda de capitalização de mercado utilizando os dados da taxa de câmbio;

Carregar os dados transformados em um CSV separado;

In [1]:
#Importando bibliotecas

!mamba install pandas==1.3.3 -y
!mamba install requests==2.26.0 -y

/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found


In [2]:
#Importando bibliotecas adicionais

import glob
import pandas as pd
from datetime import datetime

In [3]:
#Download do dataset

!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-09-25 12:54:05--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-09-25 12:54:05 (512 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2023-09-25 12:54:06--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/excha

# **Extract**

In [4]:
#Função para extrair ficheiro JSON

def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [5]:
#Função de extração que encontra o ficheiro JSON e chama a função criada acima para extrair os dados

def extract():
    extracted_data = pd.DataFrame(columns=['Name','Market Cap (US$ Billion)'])

    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)

    return extracted_data

In [6]:
#Carregando ficheiro csv com a taxa de câmbio para as libras esterlinas(GBP) e armazenando na variável exchange_rate para posterior conversão dos dados

df = pd.read_csv('exchange_rates.csv', index_col=0)
exchange_rate = df.loc['GBP','Rates']
exchange_rate

0.7323984208000001

# **Transform**

In [7]:
#Função de transformação que altera os dados da coluna Market Cap (US$ Billion) de USD para GBP, renomeia e arredonda em 3 casas decimais

def transform(data):
        data['Market Cap (US$ Billion)'] = (data['Market Cap (US$ Billion)']*exchange_rate).round(3)
        data = data.rename(columns={'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'})
        return data

# **Load**

In [8]:
#Função que carrega os dados processados em um arquivo csv

def load(bank_market_cap_gbp,data):
    data.to_csv("bank_market_cap_gbp", index=False)

### **Logging**

In [9]:
#Função log para registar o processo ETL

def log(message):
    timestamp_format = '%H:%M:%S-%d/%h/%Y'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("bank_market_cap_gbp.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

# **Execução do processo ETL**

In [10]:
#Início do processo ETL

log("ETL Job Started")

In [11]:
#Início do processo de extração

log("Extract phase Started")

In [12]:
#Extraindo dados

extracted_data = extract()

extracted_data.head()

<ipython-input-5-747bf04970a0>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)


,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [13]:
#Término da extração

log("Extract phase Ended")

In [14]:
#Início do processo de transformação

log("Transform phase Started")

In [15]:
#Transformando dados

transformed_data = transform(extracted_data)

transformed_data.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


In [16]:
#Término da transformação

log("Transform phase Ended")

In [17]:
#Início do processo de caregamento

log("Load phase Started")

In [18]:
#Carregando dados

load('bank_market_cap_gbp',transformed_data)

In [19]:
#Término do carregamento

log("Load phase Ended")

In [20]:
#Fim do processo ETL

log("ETL Job Ended")